## Installing the essential modules

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [119]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import geopandas
import csv
from SPARQLWrapper import SPARQLWrapper, JSON
from string import Template

## Dataset of 3275 Tweets form the months September and October 2015 of refugees and migrants crossed the border to Hungary, Austria and Germany. Data collected from https://zenodo.org/record/2536590#.X8t9jWj0lhE 

In [4]:
#raw json data 
tweet_data = pd.read_json(r"C:\Users\insan\OneDrive\Desktop\task_initial\initial_task_text\Tweet_data\all_data.json")

# View the  Tweet data


In [5]:
data_1 = pd.DataFrame([]) #Storing Tweet in DataFrame
for key, value in tweet_data.items():
    if key == "text" :
        #select the tweet data from json 
        data_1 = data_1.append(pd.DataFrame({'Tweet': value}))
        
#removing http URL from raw tweet         
data_1['Tweet'] = data_1['Tweet'].replace(r'http\S+', '', regex=True)

#Displaying top 5 row of the DataFrame
print(data_1.head(10))

                                               Tweet
0  WTF, die grüne Grenze zwischen Österreich und ...
1  #refugeeswelcome hashtag calls for change: CNN...
2  RT @nytimes: Nearly 5000 migrants arrived in A...
3  RT @ECentauri: 20,000 MIGRANTS HAVE ENTERED CR...
4  Hundreds of migrants leave Budapest station an...
5  RT @AfricasaCountry: While you're praising Ger...
6  RT @kongoecho: Historic: Germany expects 50.00...
7  RT hrw: RT bouckap: Now that #Germany has agre...
8  RT @PierreCrom: Border with Serbia in #Roszke ...
9  @CunningDessert Gah that's so sad!!! 4 million...


Total row in dataset: 3725

# Manual location annotation of 50 tweet

In [6]:
tweet_location= pd.read_csv(r"C:\Users\insan\OneDrive\Desktop\task_initial\initial_task_text\Tweet_data\raw_tweet.csv",sep=";",index_col=0)
print(tweet_location)

                                                Tweet  \
id                                                      
1   WTF, die grüne Grenze zwischen Österreich und ...   
2   #refugeeswelcome hashtag calls for change: CNN...   
3   RT @nytimes: Nearly 5000 migrants arrived in A...   
4   RT @ECentauri: 20,000 MIGRANTS HAVE ENTERED CR...   
5   Hundreds of migrants leave Budapest station an...   
6   RT @AfricasaCountry: While you're praising Ger...   
7   RT @kongoecho: Historic: Germany expects 50.00...   
8   RT hrw: RT bouckap: Now that #Germany has agre...   
9   RT @PierreCrom: Border with Serbia in #Roszke ...   
10  @CunningDessert Gah that's so sad!!! 4 million...   
11  RT @BreitbartLondon: The Truth: Leaked Documen...   
12  Germany has talked about taking upward of 800,...   
13  RT @business: Hundreds of refugees arrive in G...   
14  Hundreds of migrants protest at Budapest stati...   
15    France announces it will take in 24,000 refu...   
16  RT @AUStockMarket: Germany 

## Getting coordinates(Lat, Long) from location name using DBpedia SPARQL


In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON

import pandas as pd

df = pd.DataFrame([]) #Storing coordinates in DataFrame
list_coordinate=[] 

#reading tweet with manual location annotataion
tweet_location= pd.read_csv(r"C:\Users\insan\OneDrive\Desktop\task_initial\initial_task_text\Tweet_data\raw_tweet.csv",sep=";",index_col=0)

for index, row in tweet_location.iterrows():

    #print(row['Location'])
    a=list(row['Location'].split(","))
    length=len(a)
    for i in a:                          #iterating depends on number of location in rows
        #print(i)
        #location=row['Location']
        location=i
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        query = Template("""
            PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
            SELECT DISTINCT ?s ?lat ?long
            WHERE { ?s rdfs:label $uri;
                    geo:lat ?lat;
                    geo:long ?long
                    } LIMIT 1
        """)
        c=location #row['Location']
        b='"'+c+'"'+'@en'                 
        #print(b)
        sparql.setQuery(query.substitute(uri=b))
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
       #print(results)
        for result in results["results"]["bindings"]:
            print(result)
       
    #Displaying lat and long from SPARQL
    
            #print(result["lat"]["value"],result["long"]["value"], result["s"]["value"])
            lat=float(result["lat"]["value"])
            
            long=float(result["long"]["value"])
            
            df = df.append(pd.DataFrame({"id":[index],"Location":c, "Lat": [lat],"Long":[long]}))
            list_coordinate+=result["lat"]["value"],result["long"]["value"]


#print(list_coordinate)
print(df)
            



{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Germany'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '51.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '9.0'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Austria'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.3333'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '13.3333'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Hungary'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '19.25'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Budapest'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSche

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Austria'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.3333'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '13.3333'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Germany'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '51.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '9.0'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Hungary'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '19.25'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Budapest'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSche

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Hungary'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '19.25'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Germany'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '51.0'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '9.0'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Austria'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '47.3333'}, 'long': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#float', 'value': '13.3333'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Germany'}, 'lat': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchem

## Storing coordinates and id which will use to map with location from annotated dataset

In [10]:
print(df.head())

   id  Location      Lat     Long
0   2   Germany  51.0000   9.0000
0   3   Austria  47.3333  13.3333
0   3   Hungary  47.0000  19.2500
0   5  Budapest  47.4925  19.0514
0   6   Germany  51.0000   9.0000


## Creating a GeoDataFrame from a DataFrame with coordinates

In [13]:
gdf= geopandas.GeoDataFrame(df,geometry=geopandas.points_from_xy(df.Lat, df.Long))

In [15]:
gdf[['id','Location','Lat','Long','geometry']]

,id,Location,Lat,Long,geometry
0,2,Germany,51.0000,9.0000,POINT (51.00000 9.00000)
0,3,Austria,47.3333,13.3333,POINT (47.33330 13.33330)
0,3,Hungary,47.0000,19.2500,POINT (47.00000 19.25000)
0,5,Budapest,47.4925,19.0514,POINT (47.49250 19.05140)
0,6,Germany,51.0000,9.0000,POINT (51.00000 9.00000)
...,...,...,...,...,...
0,45,Austria,47.3333,13.3333,POINT (47.33330 13.33330)
0,47,Germany,51.0000,9.0000,POINT (51.00000 9.00000)
0,47,Hamburg,53.5653,10.0014,POINT (53.56530 10.00140)
0,48,Hamburg,53.5653,10.0014,POINT (53.56530 10.00140)
